In [1]:
import os
os.chdir('../')
%pwd

'd:\\A - My Projects\\A - MLOps\\Flower-Gift-Helper'

# Entity

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    trained_model_path: Path
    test_dir: Path
    params_input_shape: list
    params_batch_size: int
    params_all: dict

# Configuration Manager

In [3]:
from flowerClassifier import logger, CONFIG_FILE_PATH, PARAMS_FILE_PATH
from flowerClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigManager():
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root_dir])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
                
        model_evaluation_config = ModelEvaluationConfig(
                trained_model_path=self.config.model_training.trained_model_path,
                test_dir=self.config.data_ingestion.data_dir,
                params_input_shape=self.params.IMAGE_SIZE,
                params_batch_size=self.params.BATCH_SIZE,
                params_all=self.params
        )
        
        return model_evaluation_config

# Component

In [5]:
import tensorflow as tf
from pathlib import Path
import mlflow
from flowerClassifier.utils.common import save_json

In [8]:
class EvaluateModel():
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        self.model_signature = None
        
    def create_test_generator(self):
        test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

        # configure data flow
        data_flow_kwargs = dict(
            target_size=self.config.params_input_shape[:2],
            batch_size=self.config.params_batch_size,
            class_mode='categorical',
            interpolation="bilinear"
        )

        self.test_generator = test_datagen.flow_from_directory(
            directory = self.config.test_dir,
            ** data_flow_kwargs
        )
        
    def create_signature(self):
        
        # Extract a single image and its label
        X_batch, y_batch = next(self.test_generator)
        X_sample=X_batch[0:1]
        y_sample=y_batch[0:1]

        # Get model predictions for the sample image
        y_pred=self.trained_model.predict(X_sample)

        # Infer the model signature using the input (X_sample) and the output (y_pred)
        self.model_signature = mlflow.models.signature.infer_signature(X_sample, y_pred)
        logger.info(f"Model signature created")
        
    def save_score(self):
        self.scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("metrics.json"), data=self.scores)
        
    def evaluate_model(self):
        self.trained_model = tf.keras.models.load_model(self.config.trained_model_path)
        self.create_test_generator()
        self.score = self.trained_model.evaluate(self.test_generator)
        logger.info(f'The accuracy of the model is: {self.score[1]*100:.2f}%')
        logger.info(f'The loss of the model is: {self.score[0]:.4f}')
        self.save_score()
        
    def log_to_mlflow(self):
        
        # Log the model with the inferred signature using MLflow
        with mlflow.start_run():
            
            # Log model and metrics to MLflow
            mlflow.log_params(self.config.params_all)
            mlflow.log_metrics(self.scores)
            
            # if signature is not created, create a signature
            if self.model_signature == None:
                self.create_signature()
                
            # Log the model
            mlflow.keras.log_model(self.trained_model, "mobilenetv2_model", signature=self.model_signature)
            logger.info("Model logged")

# Pipeline

In [10]:
config = ConfigManager()
model_evaluation_config = config.get_model_evaluation_config()
evaluate_model = EvaluateModel(config=model_evaluation_config)
evaluate_model.evaluate_model()
evaluate_model.log_to_mlflow()

[2024-09-09 12:01:10,625 | INFO] common: yaml file: constant\config.yaml loaded successfully
[2024-09-09 12:01:10,627 | INFO] common: yaml file: params.yaml loaded successfully
[2024-09-09 12:01:10,628 | INFO] common: created directory at: artifacts
Found 2746 images belonging to 5 classes.


d:\A - My Projects\A - MLOps\Flower-Gift-Helper\.conda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 32s 697ms/step - accuracy: 0.9074 - loss: 0.2386
[2024-09-09 12:01:43,275 | INFO] 4184225023: The accuracy of the model is: 91.33%
[2024-09-09 12:01:43,276 | INFO] 4184225023: The loss of the model is: 0.2400
[2024-09-09 12:01:43,277 | INFO] common: json file saved at: metrics.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 968ms/step
[2024-09-09 12:01:44,441 | INFO] 4184225023: Model signature created
[2024-09-09 12:01:51,405 | INFO] 4184225023: Model logged
